# Poelis Python SDK Tutorial

Welcome to the Poelis Python SDK tutorial! This notebook will guide you through setting up and using the SDK to interact with the Poelis API.

### What is Poelis?

Poelis is a platform for managing hierarchical data structures, products, items, and their properties. The Python SDK provides a convenient way to interact with the Poelis API from your Python applications.

### Table of Contents

1. [Installation](#1-installation)
2. [IDE Compatibility & Autocomplete](#2-ide-compatibility--autocomplete)
3. [Authentication & Client Setup](#3-authentication--client-setup)
4. [Browser Interface](#4-browser-interface)
5. [Product Versions](#5-product-versions)
6. [Writing Property Values](#6-writing-property-values)



## 1. Installation

### Requirements
- Python >= 3.11
- Access to a Poelis API endpoint
- Valid SDK API key (user-bound; org/workspaces derived server-side)


In [ ]:
!pip install -U poelis-sdk

## 2. IDE Compatibility & Autocomplete

The Poelis SDK works great in both **VS Code** and **PyCharm**, but there are some differences in how autocomplete behaves:

### VS Code (Recommended for Best Experience)
- ✅ **Perfect autocomplete**: Dynamic attributes work automatically
- ✅ **No setup required**: Just start typing and press TAB
- ✅ **Fast and responsive**: Real-time completion suggestions
- ✅ **Works in Jupyter notebooks**: Full autocomplete support

### PyCharm (Good, but with limitations)
- ⚠️ **Limited autocomplete**: Uses static analysis, not runtime introspection
- ⚠️ **Requires priming**: Call `names()` at each level for autocomplete
- ✅ **Code works perfectly**: All functionality works, just autocomplete is limited
- ✅ **Workarounds available**: Interactive selectors and helper functions

**Recommendation**: Use **VS Code** for the best autocomplete experience, especially in Jupyter notebooks.

### Notebooks vs Python Scripts

**In Jupyter Notebooks**: The browser interface works great! You get autocomplete (especially in VS Code) and can easily explore your data interactively using dot notation like `poelis.demo_workspace.demo_product`.

**In Python Scripts**: For better reliability and performance, it's recommended to copy IDs from the webapp and use them directly.


## 3. Authentication & Client Setup

Before you can use the SDK, you need to set up authentication and initialize the client.

### Getting Your Credentials

1. **Navigate to Organization Settings → API Keys** in the Poelis web interface
2. **Create a new SDK API key**
3. **Important**: SDK keys are **user-bound**; organization and workspace access are derived on the server from the Poelis user behind that key.
4. With this key, you only see the workspaces your Poelis user is assigned to in that org.

In [ ]:
# Import the SDK
from poelis_sdk import PoelisClient

# Example: Initialize client (replace with your actual API key)

poelis_client = PoelisClient(
    api_key="your_api_key_here",
)

poelis = poelis_client.browser

## 4. Browser Interface

Navigate your data using dot notation: `poelis.<workspace>.<product>.<item>.<property>`

**IDE Note**: VS Code has perfect autocomplete. PyCharm users should use `list_*().names` to discover available names.

### Available Functions

#### 1. `list_workspaces()` - List all workspaces

#### 2. `list_products()` - List products in a workspace

#### 3. `list_items()` - List items in a product/item

#### 4. `list_properties()` - List properties of an item

#### 5. `get_property(readable_id)` - Get property by readableId


## Property Change Detection

The SDK can automatically warn you when property values change between script/notebook runs. This is useful when you're using property values for calculations and want to be notified if a colleague changes them in the webapp.


In [ ]:
# Example: list_workspaces()
workspaces = poelis.list_workspaces().names
print("Available workspaces:", workspaces)

In [ ]:
# Example: list_products()
ws = poelis.demo_workspace  # Replace with your workspace name
products = ws.list_products().names
print("Products in workspace:", products)

Error: 

In [ ]:
# Example: list_items()
prod = ws.demo_product
items = prod.list_items().names
print(items)  # ['item1', 'item2', ...]

# At item level: lists only child items (not properties)
item = prod.demo_item
child_items = item.list_items().names

In [ ]:
# Example: list_properties()
item = prod.demo_item  # Replace with your item name
properties = item.list_properties().names
print("Properties:", properties)

In [ ]:
# Example: get_property(readable_id)
# Note: get_property() is only available on item nodes. Navigate to an item first.
property_readable_id = "demo_property_mass"  # Replace with your property's readableId
# Access item from baseline, then get property from the item
item = prod.baseline.demo_item  # or prod.baseline["item_name"]
prop = item.get_property(property_readable_id)
print(f"Value: {prop.value}, Category: {prop.category}, Unit: {prop.unit}")

## 5. Product Versions

Poelis supports **product versioning**, which allows you to create frozen snapshots of your product data at specific points in time. This is useful for tracking changes, maintaining historical records, and ensuring reproducibility.

### Understanding Versions

- **Draft**: The current working state of your product (not yet versioned)
- **Versioned snapshots**: Frozen snapshots of your product at specific version numbers (v1, v2, v3, etc.)
- **Baseline**: The latest versioned snapshot (highest version number)

**Important**: When you access a product without specifying a version (e.g., `product.my_item`), the SDK defaults to the **baseline** version (the latest versioned snapshot). If no versions exist yet, it falls back to the draft.

### Available Functions

#### 1. `list_product_versions()` - List all versions of a product

#### 2. `get_version()` - Access a version of a product by its title

In [ ]:
# Example: List product versions
product = poelis.demo_workspace.demo_product  # Replace with your product

# List all available versions (including draft)
versions = product.list_product_versions().names
print("Available versions:", versions)

In [ ]:
# Example: Access different versions via browser interface
product = poelis.demo_workspace.demo_product  # Replace with your product

# Access draft version
draft = product.draft
print("Draft version items:", draft.list_items().names[:5])  # First 5 items

# Access baseline (latest versioned snapshot)
baseline = product.baseline
print("Baseline version items:", baseline.list_items().names[:5])

# Access a specific version (e.g., v1)
if "v1" in product.list_product_versions().names:
    v1 = product.v1
    print("Version 1 items:", v1.list_items().names[:5])

# Access a version by name
version_prod_2025 = product.get_version("PROD 2025")
print("Version items:", version_prod_2025.list_items().names[:5])

## 6. Writing Property Values

The SDK allows you to update property values for **draft versions** of products. This is useful when you need to modify property data programmatically.

**Permissions**: Write operations require **EDITOR** role for the workspace or product. Only properties in the **draft** version can be updated. 

#### `change_property(value, [title='Title of the change'], [description='Description of the change'])`


### Example: Update Numeric Property

Numeric properties can accept single numbers, arrays, or matrices:


In [ ]:
# Example: Update a numeric property (single value)
mass = poelis.demo_workspace.demo_product.draft.demo_item.demo_property_mass  # Replace with your property name

# Print the old value
print(f"Old value: {mass.value}")

# Update with a single numeric value
mass.change_property(123.45, title='Updated mass', description='Changed mass value for testing')

# Verify the change
print(f"New value: {mass.value}")


In [ ]:
# Example: Update numeric property with matrix
matrix_prop = poelis.demo_workspace.demo_product.draft.demo_item.demo_property_matrix

matrix_prop.change_property([[1, 2, 3], [4, 5, 6]])

### Example: Update Text Property


In [ ]:
# Example: Update a text property
text_prop = ws.demo_product.draft.demo_item.demo_property_string

text_prop.change_property("New string text")

print(f"New text value: {text_prop.value}")
